In [ ]:
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
import numpy as np
import math
import time
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import regularizers
import re

AUTO = tf.data.experimental.AUTOTUNE

print("Tensorflow version " + tf.__version__)

In [ ]:
# code in this cell is referenced from https://www.kaggle.com/code/philculliton/a-simple-petals-tf-2-2-notebook/notebook
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

In [ ]:
# code in this cell is referenced from https://www.kaggle.com/code/philculliton/a-simple-petals-tf-2-2-notebook/notebook
IMAGE_SIZE = [192, 192] # at this size, a GPU will run out of memory. Use the TPU
EPOCHS = 25
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
HEIGHT = 192
WIDTH = 192
CHANNELS = 3
GCS_PATH = GCS_DS_PATH + '/tfrecords-jpeg-192x192'
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec') 

CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']  




In [ ]:
# code in this cell is referenced from https://www.kaggle.com/code/philculliton/a-simple-petals-tf-2-2-notebook/notebook
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)


In [ ]:
# code in this cell is referenced from https://www.kaggle.com/code/philculliton/a-simple-petals-tf-2-2-notebook/notebook
def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames,num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset


In [ ]:
# code in this cell is referenced from https://www.kaggle.com/code/philculliton/a-simple-petals-tf-2-2-notebook/notebook
def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    #dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

def get_validation_dataset():
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=False)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [ ]:
# code in this cell is referenced from https://www.kaggle.com/code/philculliton/a-simple-petals-tf-2-2-notebook/notebook
def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec
    # files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

training_dataset = get_training_dataset()
validation_dataset = get_validation_dataset()
test_dataset = get_test_dataset(ordered=True)

In [ ]:
# code in this cell is referenced from https://www.kaggle.com/code/philculliton/a-simple-petals-tf-2-2-notebook/notebook
np.set_printoptions(threshold=15, linewidth=80)

print("Training data shapes:")
for image, label in training_dataset.take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Training data label examples:", label.numpy())

print("Test data shapes:")
for image, idnum in test_dataset.take(3):
    print(image.numpy().shape, idnum.numpy().shape)
print("Test data IDs:", idnum.numpy().astype('U')) # U=unicode string

In [ ]:
# code in this cell is referenced from https://www.kaggle.com/code/philculliton/a-simple-petals-tf-2-2-notebook/notebook
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])
    


In [ ]:
#my code
model_names = []
model_training_time = []
def display_training_time_plot(model_names, model_training_time):
    plt.bar(model_names, model_training_time)
    plt.xlabel('models')
    plt.ylabel('training time')
    plt.xticks(rotation=90)
    plt.title("Training Time of  different models")
    plt.show()

train_accuracy = []
val_accuracy = []


def display_accuracy_plot(model_names, train_accuracy, val_accuracy):
    fig, ax = plt.subplots()
    ax.plot(model_names, train_accuracy)
    ax.plot(model_names, val_accuracy)
    plt.xticks(rotation=90)
    ax.legend(['train', 'valid.'])
    plt.show()

In [ ]:
#my code; the model defination is original
with strategy.scope():    
    
    # Define the model as a sequential sequence of layers
    model1 = Sequential()

    # Define convolutional layers
    model1.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(*IMAGE_SIZE, 3)))
    model1.add(MaxPooling2D((2, 2)))

    model1.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model1.add(MaxPooling2D((2, 2)))
    
    model1.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model1.add(MaxPooling2D((2, 2)))


    # Define classification layers
    model1.add(Flatten())
    model1.add(Dense(1024, activation='relu'))
    model1.add(Dropout(0.5))
    model1.add(Dense(len(CLASSES), activation='softmax'))

    # Print a summary of the model architecture
    print(model1.summary())

      
model1.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)


start_time = time.time()
model1_history = model1.fit(training_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, 
          epochs=EPOCHS, 
          validation_data=validation_dataset)
end_time = time.time()
model_training_time.append(end_time-start_time)
model_names.append('model1')

display_training_curves(
    model1_history.history['loss'],
    model1_history.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    model1_history.history['sparse_categorical_accuracy'],
    model1_history.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)

train_accuracy.append(model1_history.history['sparse_categorical_accuracy'][-1])
val_accuracy.append(model1_history.history['val_sparse_categorical_accuracy'][-1])

The training accuracy is 97.43% and validation accuracy is 33.35%, which is a strong indication that the model is overfitted. 
https://www.analyticsvidhya.com/blog/2020/09/overfitting-in-cnn-show-to-treat-overfitting-in-convolutional-neural-networks/
Overfitting can be resolved using following 
* Regularization
* Weight Initialization
* Dropout Regularization
* Weight Constraints

Below I have added L2 Regularization to the hidden layers 2 and 3

In [ ]:
#my code; the model defination is original
with strategy.scope():    
    
    # Define the model as a sequential sequence of layers
    model1_regularization = Sequential()

    # Define convolutional layers
    model1_regularization.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(*IMAGE_SIZE, 3)))
    model1_regularization.add(MaxPooling2D((2, 2)))

    model1_regularization.add(Conv2D(256, (3, 3), activation='relu',kernel_regularizer=regularizers.l2(l=0.01), padding='same'))
    model1_regularization.add(MaxPooling2D((2, 2)))
    
    model1_regularization.add(Conv2D(512, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l=0.01),padding='same'))
    model1_regularization.add(MaxPooling2D((2, 2)))


    # Define classification layers
    model1_regularization.add(Flatten())
    model1_regularization.add(Dense(1024, activation='relu'))
    model1_regularization.add(Dropout(0.5))
    model1_regularization.add(Dense(len(CLASSES), activation='softmax'))

    # Print a summary of the model architecture
    print(model1_regularization.summary())

      
model1_regularization.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)


start_time = time.time()
model1_regularization_history = model1_regularization.fit(training_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, 
          epochs=EPOCHS, 
          validation_data=validation_dataset)
end_time = time.time()
model_training_time.append(end_time-start_time)
model_names.append('model1_regularization')

display_training_curves(
    model1_regularization_history.history['loss'],
    model1_regularization_history.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    model1_regularization_history.history['sparse_categorical_accuracy'],
    model1_regularization_history.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)

train_accuracy.append(model1_regularization_history.history['sparse_categorical_accuracy'][-1])
val_accuracy.append(model1_regularization_history.history['val_sparse_categorical_accuracy'][-1])

Addition of L2 Regularizer to layer 2 and layer 3 led to the increase in the validation accuracy. Althought there is an increase in accuracy for validation data set. It still appears that the model is overfitting to the training data.


Next we build a more complex CNN model with more convolution layers.

In [ ]:
#my code; the model defination is original
with strategy.scope():    
    
    # Define the model as a sequential sequence of layers
    model2 = Sequential()

    # Define convolutional layers
    model2.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(*IMAGE_SIZE, 3)))
    model2.add(MaxPooling2D((2, 2)))

    model2.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model2.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model2.add(MaxPooling2D((2, 2)))
    
    model2.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model2.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model2.add(MaxPooling2D((2, 2)))
    
    model2.add(Conv2D(1024, (3, 3), activation='relu', padding='same'))
    model2.add(Conv2D(1024, (3, 3), activation='relu', padding='same'))
    model2.add(MaxPooling2D((2, 2)))
    
    # Define classification layers
    model2.add(Flatten())
    model2.add(Dense(1024, activation='relu'))
    model2.add(Dropout(0.4))
    model2.add(Dense(512, activation='relu'))
    model2.add(Dropout(0.2))
    model2.add(Dense(len(CLASSES), activation='softmax'))

    # Print a summary of the model architecture
    print(model2.summary())

      
model2.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)


start_time = time.time()
model2_history = model2.fit(training_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, 
          epochs=EPOCHS, 
          validation_data=validation_dataset)
end_time = time.time()
model_training_time.append(end_time-start_time)
model_names.append('model2')

display_training_curves(
    model2_history.history['loss'],
    model2_history.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    model2_history.history['sparse_categorical_accuracy'],
    model2_history.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)

train_accuracy.append(model2_history.history['sparse_categorical_accuracy'][-1])
val_accuracy.append(model2_history.history['val_sparse_categorical_accuracy'][-1])

In [ ]:
#my code; the model defination is original
with strategy.scope():    
    
    # Define the model as a sequential sequence of layers
    model2_regularization = Sequential()

    # Define convolutional layers
    model2_regularization.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(*IMAGE_SIZE, 3)))
    model2_regularization.add(MaxPooling2D((2, 2)))

    model2_regularization.add(Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l=0.01), padding='same'))
    model2_regularization.add(Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l=0.01), padding='same'))
    model2_regularization.add(MaxPooling2D((2, 2)))
    
    model2_regularization.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model2_regularization.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model2_regularization.add(MaxPooling2D((2, 2)))
    
    model2_regularization.add(Conv2D(1024, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l=0.01),padding='same'))
    model2_regularization.add(Conv2D(1024, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l=0.01),padding='same'))
    model2_regularization.add(MaxPooling2D((2, 2)))
    


    # Define classification layers
    model2_regularization.add(Flatten())
    model2_regularization.add(Dense(1024, activation='relu'))
    model2_regularization.add(Dropout(0.4))
    model2_regularization.add(Dense(512, activation='relu'))
    model2_regularization.add(Dropout(0.2))
    model2_regularization.add(Dense(len(CLASSES), activation='softmax'))

    # Print a summary of the model architecture
    print(model2_regularization.summary())

      
model2_regularization.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)


start_time = time.time()
model2_regularization_history = model2_regularization.fit(training_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, 
          epochs=EPOCHS, 
          validation_data=validation_dataset)
end_time = time.time()
model_training_time.append(end_time-start_time)
model_names.append('model2_regularization')

display_training_curves(
    model2_regularization_history.history['loss'],
    model2_regularization_history.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    model2_regularization_history.history['sparse_categorical_accuracy'],
    model2_regularization_history.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)

train_accuracy.append(model2_regularization_history.history['sparse_categorical_accuracy'][-1])
val_accuracy.append(model2_regularization_history.history['val_sparse_categorical_accuracy'][-1])

Next up, we try to fit VGG16 model for flower classification with and without the pretraining.

In [ ]:
# code in this cell is referenced from https://medium.com/geekculture/boost-your-image-classification-model-with-pretrained-vgg-16-ec185f763104#:~:text=The%20VGG16%20model%20is%20a,and%203%20fully%20connected%20layers.&text=The%20pretrained%20VGG16%20model%20is,a%20wide%20range%20of%20features
with strategy.scope():    
    
    #VGG16 
    # Define the model as a sequential sequence of layers
    vgg16_model = Sequential()

    # Define convolutional layers
    vgg16_model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(*IMAGE_SIZE, 3)))
    vgg16_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    vgg16_model.add(MaxPooling2D((2, 2)))

    vgg16_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    vgg16_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    vgg16_model.add(MaxPooling2D((2, 2)))

    vgg16_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    vgg16_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    vgg16_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    vgg16_model.add(MaxPooling2D((2, 2)))

    vgg16_model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    vgg16_model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    vgg16_model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    vgg16_model.add(MaxPooling2D((2, 2)))

    vgg16_model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    vgg16_model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    vgg16_model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    vgg16_model.add(MaxPooling2D((2, 2)))

    # Define classification layers
    vgg16_model.add(Flatten())
    vgg16_model.add(Dense(4096, activation='relu'))
    vgg16_model.add(Dropout(0.5))
    vgg16_model.add(Dense(4096, activation='relu'))
    vgg16_model.add(Dropout(0.5))
    vgg16_model.add(Dense(len(CLASSES), activation='softmax'))

    # Print a summary of the model architecture
    print(vgg16_model.summary())

      
vgg16_model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)


start_time = time.time()
vgg16_model_history = vgg16_model.fit(training_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, 
          epochs=EPOCHS, 
          validation_data=validation_dataset)
end_time = time.time()
model_training_time.append(end_time-start_time)
model_names.append('vgg16_model')

display_training_curves(
    vgg16_model_history.history['loss'],
    vgg16_model_history.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    vgg16_model_history.history['sparse_categorical_accuracy'],
    vgg16_model_history.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)

train_accuracy.append(vgg16_model_history.history['sparse_categorical_accuracy'][-1])
val_accuracy.append(vgg16_model_history.history['val_sparse_categorical_accuracy'][-1])

In [ ]:
# code in this cell is referenced from https://www.kaggle.com/code/philculliton/a-simple-petals-tf-2-2-notebook/notebook
with strategy.scope():    
    pretrained_VGG16 = tf.keras.applications.VGG16(
        weights='imagenet',
        include_top=False ,
        input_shape=[*IMAGE_SIZE, 3]
    )
    pretrained_VGG16.trainable = False
    
    pretrained_VGG16_model = tf.keras.Sequential([
        # To a base pretrained on ImageNet to extract features from images...
        pretrained_VGG16,
        # ... attach a new head to act as a classifier.
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])
    print(pretrained_VGG16_model.summary())

pretrained_VGG16_model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

start_time = time.time()
pretrained_VGG16_model_history = pretrained_VGG16_model.fit(training_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, 
          epochs=EPOCHS, 
          validation_data=validation_dataset)
end_time = time.time()
model_training_time.append(end_time-start_time)
model_names.append('pretrained_VGG16_model')

display_training_curves(
    pretrained_VGG16_model_history.history['loss'],
    pretrained_VGG16_model_history.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    pretrained_VGG16_model_history.history['sparse_categorical_accuracy'],
    pretrained_VGG16_model_history.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)
train_accuracy.append(pretrained_VGG16_model_history.history['sparse_categorical_accuracy'][-1])
val_accuracy.append(pretrained_VGG16_model_history.history['val_sparse_categorical_accuracy'][-1])

In [ ]:
#my code
display_training_time_plot(model_names, model_training_time)

In [ ]:
#my code
display_accuracy_plot(model_names, train_accuracy, val_accuracy)

In [ ]:
#code in this cell is referenced from https://www.kaggle.com/code/philculliton/a-simple-petals-tf-2-2-notebook/notebook
#saving predictions from model2
test_images_ds = test_dataset.map(lambda image, idnum: image)
probabilities = model1_regularization.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)



In [ ]:
#code in this cell is referenced from https://www.kaggle.com/code/philculliton/a-simple-petals-tf-2-2-notebook/notebook
test_ids_ds = test_dataset.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')
print('submission.csv file saved')